In [ ]:
!nvidia-smi

Thu Jan  7 01:36:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 14662 (delta 1), reused 1 (delta 0), pack-reused 14657
Receiving objects: 100% (14662/14662), 13.27 MiB | 24.88 MiB/s, done.
Resolving deltas: 100% (9976/9976), done.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!echo "Sheep" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/Sheep/' > data/obj.data
!mkdir data/obj

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-01-07 01:38:24--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  3.57MB/s    in 47s     

2021-01-07 01:39:12 (3.27 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!unzip /mydrive/Sheep/images.zip -d data/obj

Archive:  /mydrive/Sheep/images.zip
   creating: data/obj/images/
  inflating: data/obj/images/000000002178.jpg  
  inflating: data/obj/images/000000002178.txt  
  inflating: data/obj/images/000000004624.jpg  
  inflating: data/obj/images/000000004624.txt  
  inflating: data/obj/images/000000004947.jpg  
  inflating: data/obj/images/000000004947.txt  
  inflating: data/obj/images/000000006336.jpg  
  inflating: data/obj/images/000000006336.txt  
  inflating: data/obj/images/000000006488.jpg  
  inflating: data/obj/images/000000006488.txt  
  inflating: data/obj/images/000000008881.jpg  
  inflating: data/obj/images/000000008881.txt  
  inflating: data/obj/images/000000009075.jpg  
  inflating: data/obj/images/000000009075.txt  
  inflating: data/obj/images/000000010539.jpg  
  inflating: data/obj/images/000000010539.txt  
  inflating: data/obj/images/000000012824.jpg  
  inflating: data/obj/images/000000012824.txt  
  inflating: data/obj/images/000000012953.jpg  
  inflating: data/obj/

In [ ]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:

import glob

images_list = glob.glob("data/obj/images/*.jpg")

print(images_list)

file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close()

['data/obj/images/000000286862.jpg', 'data/obj/images/000000129715.jpg', 'data/obj/images/000000581881.jpg', 'data/obj/images/000000350363.jpg', 'data/obj/images/000000379508.jpg', 'data/obj/images/000000281301.jpg', 'data/obj/images/000000008881.jpg', 'data/obj/images/000000555738.jpg', 'data/obj/images/000000383368.jpg', 'data/obj/images/000000205505.jpg', 'data/obj/images/000000518816.jpg', 'data/obj/images/000000320445.jpg', 'data/obj/images/000000242325.jpg', 'data/obj/images/000000490187.jpg', 'data/obj/images/000000090317.jpg', 'data/obj/images/000000477679.jpg', 'data/obj/images/000000115372.jpg', 'data/obj/images/000000424171.jpg', 'data/obj/images/000000245771.jpg', 'data/obj/images/000000413880.jpg', 'data/obj/images/000000373196.jpg', 'data/obj/images/000000276639.jpg', 'data/obj/images/000000006336.jpg', 'data/obj/images/000000548951.jpg', 'data/obj/images/000000479469.jpg', 'data/obj/images/000000201188.jpg', 'data/obj/images/000000094293.jpg', 'data/obj/images/0000002113

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Görüntülenen çıkış son 5000 satıra kısaltıldı.
 total_bbox = 1482558, rewritten_bbox = 0.890488 % 

 3926: 0.997966, 0.900028 avg loss, 0.001000 rate, 10.231344 seconds, 251264 images, 0.294673 hours left
Loaded: 0.000046 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.777965), count: 15, class_loss = 0.749933, iou_loss = 1.055897, total_loss = 1.805830 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.815146), count: 18, class_loss = 2.083537, iou_loss = 0.546253, total_loss = 2.629790 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.848518), count: 3, class_loss = 0.407325, iou_loss = 0.080167, total_loss = 0.487492 
 total_bbox = 1482594, rewritten_bbox = 0.890466 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.842251), count: 8, class_loss = 0.678701, iou_loss = 0.394502, total_loss = 1.073203 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 